In [25]:
import ee
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import osmnx as ox

In [2]:
#authenticate and initialize ee
ee.Authenticate()
ee.Initialize(project = '')

In [3]:
#link the notebook to the directory of source file used for the geometry
folder = 'drive/MyDrive/porto_study_extent'
shp = 'porto-polygon.shp'

#check if the path exists
if not os.path.exists:
  os.mkdir(folder)

#now join the folder to shapefile path
file_path = os.path.join(folder, shp)

#read the filepath
if os.path.exists(file_path):
  aoi = gpd.read_file(file_path)
  print('Shapefile accessed successfully---------------------------')

else:
  print('Doublecheck path')

#view properties
print(type(aoi))
print(aoi.geometry)

Shapefile accessed successfully---------------------------
<class 'geopandas.geodataframe.GeoDataFrame'>
0    POLYGON ((-52.07086 -29.64306, -51.00342 -29.3...
Name: geometry, dtype: geometry


In [4]:
# Convert the GeoDataFrame to an GEE object
roi = geemap.geopandas_to_ee(aoi)
print(type(roi))


<class 'ee.featurecollection.FeatureCollection'>


In [5]:
#loading of images
#preflood date
fro  = '2023-01-01'
to = '2024-02-14'

#flood date
start = '2024-04-24'
stop = '2024-05-07'

#create before flood mosaic of the target area
before = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)).filterDate(fro, to).filterBounds(roi).mosaic().clip(roi)

#after flood
after = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40)).filterDate(start, stop).filterBounds(roi).mosaic().clip(roi);


# Create an interactive map
Map1 = geemap.Map(center=[-29.8, -51.45], zoom = 10)
Map1.add_basemap("ROADMAP")


In [6]:
# Visualize the image before the flood
s2_Viz = {"min":0.0, "max":3000, "bands":["B4", "B3", "B2"]}
Map1.addLayer(before, s2_Viz, "before flood")
Map1

# Visualize the image after the flood
s2_Viz = {"min":0.0, "max":3000, "bands":["B4", "B3", "B2"]}
Map1.addLayer(after, s2_Viz, "after flood")
Map1

Map(center=[-29.8, -51.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

FLOOD EXTENT MAPPING USING MNDWI

In [7]:
# Compute the MNDWI before and after flood
before_mndwi = before.normalizedDifference(['B11', 'B3'])
after_mndwi = after.normalizedDifference(['B11', 'B3'])

#add to map
Map1.addLayer(before_mndwi, {min: -1, max: 1, "palette": ['blue', 'red', 'yellow']}, 'Before MNDWI')
Map1.addLayer(after_mndwi, {min: -1, max: 1, "palette": ['blue', 'red', 'yellow']}, 'After MNDWI')

NOW IDENTIFY THE FLOOD EXTENT BY SETTING A THRESHOLD

In [8]:
#set a threshold for flood extent
thres = -1.5;
#now get difference to cap
difference = after_mndwi.divide(before_mndwi);
diffThres = difference.lt(thres).rename('Water').selfMask();

MASK OUT AREAS OF PERMANENT WATER

In [9]:
#mask out areas of permanent water
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater");
permanentWater = gsw.select('seasonality').gte(5).clip(roi);

flooded = diffThres.where(permanentWater, 0).selfMask();
# Smooth the flooded extent layer to get a refined
threshold = 6
connection = flooded.connectedPixelCount(26)
flooded = flooded.updateMask(connection.gt(threshold))
# Convert the binary image to integer with values 0 and 1
flooded_int = flooded.multiply(255).toInt()

perm_water = permanentWater.selfMask()

Map1.addLayer(permanentWater.selfMask(), {min:0, max:1, "palette": ['#2D9AFA']}, 'Permanent Water')

Map1.addLayer(flooded, {"palette": ['#FF0000']}, 'Flooded Areas')
Map1



Map(bottom=154122.0, center=[-29.8, -51.45], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

**Visualization of flooded and permanent water**


In [10]:
Map = geemap.Map(center=[-29.8, -51.45], zoom = 10)
Map.add_basemap("ROADMAP")
Map.addLayer(permanentWater.selfMask(), {min:0, max:1, "palette": ['#2D9AFA']}, 'Permanent Water')
Map.addLayer(flooded, {"palette": ['blue']}, 'Flooded Areas')
Map

Map(center=[-29.8, -51.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [23]:
#create a function that exports gee layer as a tiff image for styling externally
def gee_to_tiff(input_layer, out_folder, out_filename):
  '''Exports a GEE layer to a geotiff and downloads into storage'''

  #check for presence of out_folder
  try:
    if not os.path.exists(out_folder):
      os.mkdir(out_folder)

    #join the folder to filename
    filename = os.path.join(out_folder, out_filename)

    #export gee to tiff
    rast = geemap.ee_export_image(
            input_layer,
            filename,
            scale = 20,
            region = vec.geometry(),
            crs= "EPSG:32722",
            file_per_band = False
        )
    return rast
  except:
     print('Error ocurred')



In [12]:
#export flooded and permanent water geotiffs
gee_to_tiff(perm_water, 'outputs', 'porto_alegre_perm.tif')
gee_to_tiff(flooded, 'outputs', 'porto_alegre_flooded.tif')